In this example, I am going to develop content based recommender engine to recommend movies based on the similarties of attibutes for  movies 

In [184]:
# import the libraries
import pandas as pd
import numpy as np
import seaborn as sn
from matplotlib import pyplot as plt
from ipywidgets import interact

In [143]:
#load the movie dataset
data = pd.read_csv('movies.csv')
data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [144]:
data.shape

(9742, 3)

In [145]:
data.sample(6)

,movieId,title,genres
7353,78467,Jonah Hex (2010),Action|Drama|Thriller|Western
5128,8195,"Avventura, L' (Adventure, The) (1960)",Drama|Mystery|Romance
5964,34536,The Great Raid (2005),Action|Drama|War
770,1012,Old Yeller (1957),Children|Drama
6344,49263,Fuck (2005),Comedy|Documentary
3030,4054,Save the Last Dance (2001),Drama|Romance


In [146]:
# lets us load the rating dataset
ratings = pd.read_csv('ratings.csv')
ratings.sample(6)

,userId,movieId,rating,timestamp
94963,599,112326,1.0,1519174755
47805,308,58559,4.0,1421373894
24356,169,585,3.0,1078285078
13240,84,922,4.0,860397228
1583,16,678,4.0,1377477470
30027,209,60684,5.0,1524522466


In [147]:
ratings.shape

(100836, 4)

In [148]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [150]:
# let us merge the two datasets

data_all = pd.merge(data,ratings, on='movieId', how='inner')
data_all.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [151]:
data_all.shape

(100836, 6)

In [152]:
# lets us remove the unwanted features

data_all = data_all.drop(['movieId','userId', 'timestamp'], axis = 1)
data_all.head()

,title,genres,rating
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0
2,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5
3,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,2.5
4,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.5


In [153]:
data_all = pd.pivot_table(data_all, index = ['title', 'genres'], aggfunc='mean')

In [154]:
data_all.reset_index(level=['title','genres'], inplace=True)
data_all.head()

,title,genres,rating
0,'71 (2014),Action|Drama|Thriller|War,4.0
1,'Hellboy': The Seeds of Creation (2004),Action|Adventure|Comedy|Documentary|Fantasy,4.0
2,'Round Midnight (1986),Drama|Musical,3.5
3,'Salem's Lot (2004),Drama|Horror|Mystery|Thriller,5.0
4,'Til There Was You (1997),Drama|Romance,4.0


In [155]:
# split the genres field and extect the year from title
data_all['y']= data_all['title'].str.split(' ')
data_all['year'] = data_all['y'].apply(lambda x:x[-1])

In [156]:
data_all.drop(['y'], axis=1)

#let us remove the brackets from the year 
data_all['year']= data_all['year'].str.strip(')')
data_all['year']= data_all['year'].str.strip('(')
data_all['year'].value_counts()

2002      311
2006      295
2001      294
2000      283
2009      282
         ... 
1919        1
Mirror      1
1915        1
5           1
1902        1
Name: year, Length: 120, dtype: int64

In [162]:
# let us clean up the year field
data_all['year'] = data_all['year'].replace(('2007-','Despot','Things','1975-1979','Road','Mirror','Cosmos',
                                             '2006–2007','Baby','Moonlight','Animals','Paterson','One','Watson','OA','2','5',''),
                                           ('2007','2016','2016','1979','2002','1996','1996','2007','1996',
                                            '1996','1996' ,'1996','1996','1996','1996','1996','1996','1996' ))
data_all['year']=data_all['year'].astype(int)
data_all.drop(['y'], axis=1, inplace=True)
data_all.head()

,title,genres,rating,year
0,'71 (2014),Action|Drama|Thriller|War,4.0,2014
1,'Hellboy': The Seeds of Creation (2004),Action|Adventure|Comedy|Documentary|Fantasy,4.0,2004
2,'Round Midnight (1986),Drama|Musical,3.5,1986
3,'Salem's Lot (2004),Drama|Horror|Mystery|Thriller,5.0,2004
4,'Til There Was You (1997),Drama|Romance,4.0,1997


In [158]:
data_all.dtypes

title      object
genres     object
rating    float64
y          object
year        int32
dtype: object

In [159]:
# let us check the min and max of the year relaesed

print("First Releas Year: ", data_all['year'].min())
print("Last Releas Year: ", data_all['year'].max())

First Releas Year:  1902
Last Releas Year:  2018


In [187]:
#!pip install mlxtend

In [161]:
# import the transaction Encoder to create one-hot encoder for genres field
from mlxtend.preprocessing import TransactionEncoder

In [169]:
genres = data_all['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns=te.columns_)
genres.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,True,False
1,False,True,True,False,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,False
4,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False


In [172]:
# Convert boolean to int
genres = genres.astype('int')
genres.head(2)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
1,0,1,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [173]:
# insert the title of movies
genres.insert(0, 'title', data_all['title'])
genres.head(2)

,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,'71 (2014),0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,0
1,'Hellboy': The Seeds of Creation (2004),0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [174]:
# let us transpose the dataset
genres = genres.transpose()
genres = genres.rename(columns = genres.iloc[0])
genres = genres.drop(genres.index[0])
genres = genres.astype(int)
genres.head()

,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
(no genres listed),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Action,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
Adventure,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Animation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Children,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [188]:
# Measure the similarity between movies based on Pearson correlation
@interact
def recommendation_movie(movie = list(genres.columns)):
    similar  = genres.corrwith(genres[movie],method='pearson')
    similar = similar.sort_values(ascending = False)
    similar = similar.iloc[1:]
    return similar.head(20)

interactive(children=(Dropdown(description='movie', options=("'71 (2014)", "'Hellboy': The Seeds of Creation (…